In [6]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions

In [23]:
@model function smoothing(n_observations, noise_mean, noise_var)
    
   x_prior_mean = datavar(Float64)
   x_prior_var  = datavar(Float64)
   
   x_prior ~ NormalMeanVariance(x_prior_mean, x_prior_var)
   
   x = randomvar(n_observations)
   y = datavar(Float64, n_observations)
   
   x[1] = x_prior
   
   for i in 2:n_observations
       x[i] ~ x[i - 1] + 1.0
       y[i] ~ x[i] + NormalMeanVariance(noise_mean, noise_var)
   end
   
   return x_prior_mean, x_prior_var, x, y
end

smoothing (generic function with 1 method)

In [30]:
noise_real_mean = 0.0
noise_real_var  = 10.0

n = 500
data = map(d -> d + rand(Normal(noise_real_mean, sqrt(noise_real_var))), collect(1:n));

In [31]:
function inference(; data, x_prior)
    n = length(data)
    
    x_prior_mean, x_prior_var, x, y = smoothing(n, noise_real_mean, noise_real_var);

    subscriptions = Vector{Teardown}(undef, n)
    marginals = Vector{Marginal}(undef, n)
    
    for i in 1:n
         @inbounds subscriptions[i] = subscribe!(getmarginal(x[i]), (m) -> marginals[i] = m)
    end
    
    update!(x_prior_mean, mean(x_prior))
    update!(x_prior_var, var(x_prior))
    
    for i in 1:n
        @inbounds update!(y[i], data[i])
    end
    
    foreach(unsubscribe!, subscriptions)
    
    return marginals
end

inference (generic function with 1 method)

In [34]:
inference(
    data = data,
    x_prior = NormalMeanVariance(0.0, 10000.0)
)

500-element Array{Marginal,1}:
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=0.9016024376586085, v=0.02004003999991985))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=1.9016024376586085, v=0.02004003999991985))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=2.901602437658609, v=0.02004003999991985))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=3.901602437658608, v=0.020040039999919854))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=4.901602437658608, v=0.02004003999991985))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=5.9016024376586085, v=0.020040039999919854))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=6.9016024376586085, v=0.020040039999919854))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=7.901602437658607, v=0.020040039999919854))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64